# Cognitive Library Explorer (CLE) — Minimal Working Skeleton

Flow:
1) Ensure output folders
2) Capture+parse pip list
3) Analyze a package (runtime import is opt-in)
4) Launch ipywidgets Explorer


In [1]:
import os
# Uncomment to match Windows design path:
# os.environ['LIB_ANA_ROOT'] = r'C:\\lib_ana'
from lib_ana.config import ensure_dirs
paths = ensure_dirs()
paths

Paths(root=WindowsPath('c:/lib_ana/lib_ana'), data=WindowsPath('c:/lib_ana/lib_ana/data'), cache=WindowsPath('c:/lib_ana/lib_ana/cache'), exports=WindowsPath('c:/lib_ana/lib_ana/exports'), logs=WindowsPath('c:/lib_ana/lib_ana/logs'))

In [2]:
import subprocess, sys
from lib_ana.io_piplist import parse_pip_list_columns
from lib_ana.cache_store import CacheStore

snapshot_id = 'snapshot_demo'
text = subprocess.check_output([sys.executable, '-m', 'pip', 'list'], text=True)
res = parse_pip_list_columns(text, snapshot_id=snapshot_id)

store = CacheStore(paths.cache)
store.write_text(text, name='pip_list', snapshot_id=snapshot_id, ext='txt')
store.write_df(res.df, name='packages_inventory', snapshot_id=snapshot_id)
print('parse errors:', res.errors)
res.df.head()

parse errors: ['cannot_parse_line: ---------------------------------- ------------ -------------------------------------------------------------------', 'cannot_parse_line: opentelemetry-semantic-conventions 0.56b0']


,package,version,editable_project_location,is_editable,snapshot_id,captured_at,source_format,raw_line
0,absl-py,2.3.1,None,False,snapshot_demo,2026-02-05T10:20:49.343463+09:00,pip_list_columns,absl-py 2.3.1
1,accelerate,1.12.0,None,False,snapshot_demo,2026-02-05T10:20:49.343463+09:00,pip_list_columns,accelerate 1.12.0
2,access-deep-analyzer,4.0.0,C:\Work\AccessDeepAnalyzer,True,snapshot_demo,2026-02-05T10:20:49.343463+09:00,pip_list_columns,access-deep-analyzer 4.0.0 ...
3,access-migration-suite,0.1.0,C:\access\src\mdb_ver\access_migration_suite,True,snapshot_demo,2026-02-05T10:20:49.343463+09:00,pip_list_columns,access-migration-suite 0.1.0 ...
4,access-migration-system,0.2.0,C:\access\src\mdb_ver\access_migration_system_v3,True,snapshot_demo,2026-02-05T10:20:49.343463+09:00,pip_list_columns,access-migration-system 0.2.0 ...


In [5]:
from v5.analyzer_v5 import V5Analyzer

distribution = 'timesfm'  # change as needed
analyzer = V5Analyzer()
api_df, errors = analyzer.analyze_distribution(distribution, enable_runtime=True)
print('analysis errors:', errors)
api_df.head()

analysis errors: []


,distribution,module,qualname,object_kind,is_public,signature_str,parameters,return_annotation,docstring,doc_summary,source_path,lineno,extraction_method,errors
0,timesfm,timesfm,timesfm.ForecastConfig,class,True,None,[],None,Options for forecasting.\n\nAttributes:\n max...,Options for forecasting.,None,None,runtime,None
1,timesfm,timesfm,timesfm.TimesFM_2p5_200M_torch,class,True,None,[],None,PyTorch implementation of TimesFM 2.5 with 200...,PyTorch implementation of TimesFM 2.5 with 200...,None,None,runtime,None
2,timesfm,timesfm.TimesFM_2p5_200M_torch,timesfm.TimesFM_2p5_200M_torch.compile,method,True,"(self, forecast_config: timesfm.configs.Foreca...","[{'name': 'self', 'annotation': None, 'default...",None,Attempts to compile the model for fast decodin...,Attempts to compile the model for fast decoding.,None,None,runtime,None
3,timesfm,timesfm.TimesFM_2p5_200M_torch,timesfm.TimesFM_2p5_200M_torch.forecast,method,True,"(self, horizon: int, inputs: list[numpy.ndarra...","[{'name': 'self', 'annotation': None, 'default...",tuple,Forecasts the time series.,Forecasts the time series.,None,None,runtime,None
4,timesfm,timesfm.TimesFM_2p5_200M_torch,timesfm.TimesFM_2p5_200M_torch.forecast_with_c...,method,True,"(self, inputs: list[typing.Sequence[float]], d...","[{'name': 'self', 'annotation': None, 'default...",None,Forecasts on a list of time series with covari...,Forecasts on a list of time series with covari...,None,None,runtime,None


In [8]:
api_df.module.unique()

array(['timesfm', 'timesfm.TimesFM_2p5_200M_torch'], dtype=object)

In [ ]:
from v5.ui_v5 import V5ExplorerUI

ui = V5ExplorerUI(api_df, distribution=distribution)
ui.display()

In [10]:
from pathlib import Path
import tempfile

# --- sample inputs ---
tmp_dir = Path(tempfile.mkdtemp())
tmp_path = str(tmp_dir / 'sample.txt')

try:
    import numpy as np
    sample_arr = np.array([1, 2, 3])
except Exception:
    sample_arr = [1, 2, 3]

try:
    import pandas as pd
    sample_df = pd.DataFrame({'a':[1,2,3]})
except Exception:
    sample_df = [{'a':1},{'a':2},{'a':3}]

# --- target callable ---
import timesfm

# NOTE: generated snippet (tool does not auto-execute anything)
result = timesfm.ForecastConfig()
print(result)


ForecastConfig(max_context=0, max_horizon=0, normalize_inputs=False, window_size=0, per_core_batch_size=1, use_continuous_quantile_head=False, force_flip_invariance=True, infer_is_positive=True, fix_quantile_crossing=False, return_backcast=False)


In [12]:
!python .\src\ui_modern.py --lib timesfm --input .\outputs\api_tables\json_api.csv


Traceback (most recent call last):
  File "c:\lib_ana\src\ui_modern.py", line 969, in <module>
    main()
  File "c:\lib_ana\src\ui_modern.py", line 958, in main
    report = generate_modern_report(
             ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\lib_ana\src\ui_modern.py", line 910, in generate_modern_report
    html_text = _build_report_html(
                ^^^^^^^^^^^^^^^^^^^
  File "c:\lib_ana\src\ui_modern.py", line 685, in _build_report_html
    const top = (n.Path ?? '').split('.')[0] || '{lib}';
                                                  ^^^
NameError: name 'lib' is not defined
